In [21]:
import warnings
warnings.filterwarnings("ignore")

from data_loading import *

from xai_agg.agg_exp import *
from xai_agg.utils import *

import pandas as pd
import numpy as np

In [22]:
dataset_name, preprocessed_data, categorical_features, X, y, X_train, X_test, y_train, y_test, clf = load_pakdd2010_data()

Accuracy: 0.7466666666666667
ROC AUC: 0.5158014399393709


In [23]:
evaluator = ExplanationModelEvaluator(clf, X_train, categorical_features)
evaluator.init()

Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2223 - val_loss: 1.0931
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - loss: 1.2125 - val_loss: 1.0488
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - loss: 1.2547 - val_loss: 1.0039
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - loss: 1.1668 - val_loss: 0.9609
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - loss: 1.1460 - val_loss: 0.9237
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - loss: 1.0393 - val_loss: 0.8933
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - loss: 0.9968 - val_loss: 0.8693
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - loss: 0.9442 - val_loss: 0.8494
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - loss: 1.0604 - val_loss: 0.8331
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - loss: 1.0386 - val_loss: 0.8196
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - loss: 0.9192 - val_loss: 0.8085
Epoch 12/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 

# Generating metric vectors and calculating correlation between them

In [24]:
# Getting a few reanknings
exps = [LimeWrapper(clf, X_train, categorical_features), ShapTabularTreeWrapper(clf, X_train, categorical_features), AnchorWrapper(clf, X_train, categorical_features)]

indexes = np.random.choice(X_test.index, 100, replace=False)
comp_arr = [] # original, new1, new2, ...

In [25]:
import concurrent.futures

def process_index(idx):
    local_comp_arr = []
    for exp in exps:
        row = []
        explanation = exp.explain_instance(X_test.loc[idx])
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="sum"))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="percentile"))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="avg"))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="inverse"))
        
        row.append(evaluator.complexity(exp, X_test.loc[idx], explanation=explanation))
        row.append(evaluator.nrc(exp, X_test.loc[idx], explanation=explanation))
        local_comp_arr.append(row)
    return local_comp_arr
    
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(process_index, indexes))

for result in results:
    comp_arr.extend(result)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [26]:
from scipy.stats import spearmanr

comp_arr = np.array(comp_arr)
# rank_algs = ["Original", "sum", "percentile", "avg", "inverse"]
rank_algs = ["NRC"]

pearson_corr = [np.corrcoef(comp_arr[:, 0], comp_arr[:, i])[0, 1] for i in range(1, comp_arr.shape[1])]
spearman_corr = [spearmanr(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]

correlation_df = pd.DataFrame({
    'Rank Algorithm': rank_algs,
    'Pearson Correlation': pearson_corr,
    'Spearman Correlation': spearman_corr
})

print(dataset_name)
print(correlation_df)

PAKDD2010
  Rank Algorithm  Pearson Correlation  Spearman Correlation
0            NRC             0.495357              0.666593
